In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

import tracker
import stats
import visualizer
import utils

import importlib

In [ ]:
#videofile = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P016an R1.mp4"
videofile = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P017me R1.mp4"
#videofile = "Comparisons/10X Ph- 9Fps Wash 1 16 P018cu R1.mp4"

# Visualize the input
cap = cv2.VideoCapture(videofile)

# Get the first frame
ret, frame = cap.read()

plt.imshow(frame)

### Feed through the tracking pipeline

In [ ]:
importlib.reload(tracker)
importlib.reload(utils)

frames = utils.loadVideo(videofile,as_gray=True)
f = tracker.determineCentroids(frames)
t = tracker.trackCentroids(f)

print(t.head())

In [ ]:
importlib.reload(tracker)

compute_segments = False
if compute_segments:
    final = tracker.segmentCells(frames, t)
else:
    final = t

print(final.head())

In [14]:
utils.saveDataFrame(final, videofile.split('.')[0] + '.csv')

### Use Visualizations

In [ ]:
importlib.reload(visualizer)

visualizer.runVisualization(videofile, final, visualization="segments")

# Run Metrics

In [3]:
#prediction = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P016an R1_predicted.csv"
#groundtruth = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P016an R1_corrected.csv"
#videofile = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P016an R1.mp4"

#prediction = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P017me R1_predicted.csv"
#groundtruth = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P017me R1_corrected.csv"
#videofile = "Comparisons/10X Ph- 9Fps Wash Dil 1 16 P017me R1.mp4"

prediction = "Comparisons/10X Ph- 9Fps Wash 1 16 P018cu R1_predicted.csv"
groundtruth = "Comparisons/10X Ph- 9Fps Wash 1 16 P018cu R1_corrected.csv"
videofile = "Comparisons/10X Ph- 9Fps Wash 1 16 P018cu R1.mp4"

# Load as Pandas DataFrame
pred = utils.loadDataFrame(prediction)
gt = utils.loadDataFrame(groundtruth)


In [ ]:
# Visualize
visualizer.runVisualization(videofile, pred, visualization="flow")

In [ ]:
visualizer.runVisualization(videofile, gt, visualization="flow")

### Unlabeled Metrics

In [ ]:
import pandas as pd

def spermPerFrame(df, return_summary=False):
    # Count occurrences of each sperm count in each frame
    frame_counts = {}
    max_sperm = df["sperm"].max()
    sperm_labels = list(range(max_sperm + 1))  # Generate labels from 0 to max_sperm

    for frame in df["frame"].unique():
        frame_counts[frame] = df[df["frame"] == frame]["sperm"].value_counts().reindex(sperm_labels).fillna(0)

    # Create a DataFrame for sperm counts per frame
    frame_summary = pd.DataFrame.from_dict(frame_counts, orient='index', columns=sperm_labels)
    frame_summary.index.name = "frame"
    frame_summary.reset_index(inplace=True)

    # Add a column for total sperms observed in each frame
    frame_summary["TotalSperms"] = frame_summary[sperm_labels].sum(axis=1)

    # Extract ["TotalSperms"] as numpy array
    array = frame_summary["TotalSperms"].values

    if return_summary:
        return array, frame_summary
    else:
        return array

def framesPerSperm(df, return_summary=False):
    sperm_counts = {}
    max_frame = df["frame"].max()
    frame_labels = list(range(max_frame + 1)) # Generate labels from 0 to max_frame

    for sperm in df["sperm"].unique():
        sperm_counts[sperm] = df[df["sperm"] == sperm]["frame"].value_counts().reindex(frame_labels).fillna(0)

    # Create a DataFrame for frame counts per sperm
    sperm_summary = pd.DataFrame.from_dict(sperm_counts, orient='index', columns=frame_labels)
    sperm_summary.index.name = "sperm"
    sperm_summary.reset_index(inplace=True)

    # Add a column for total sperms observed in each frame
    sperm_summary["TotalFrames"] = sperm_summary[frame_labels].sum(axis=1)

    # Remove sperms that are not observed in any frame
    sperm_summary = sperm_summary[sperm_summary["TotalFrames"] > 0]

    # Extract ["TotalFrames"] as numpy array
    array = sperm_summary["TotalFrames"].values

    if return_summary:
        return array, sperm_summary
    else:
        return array


pred_spf = spermPerFrame(pred)
gt_spf = spermPerFrame(gt)

#print(pred_frames)
#print(gt_frames)

pred_fps = framesPerSperm(pred)
gt_fps = framesPerSperm(gt)

#print(pred_sperm)
#print(gt_sperm)

max_sperm = int(max(max(pred_spf), max(gt_spf)))

plt.hist(pred_spf, bins=range(0, max_sperm, 1), alpha=0.5, color='red', label='Predicted')
plt.hist(gt_spf, bins=range(0, max_sperm, 1), alpha=0.5, color='blue',label='Ground Truth')
plt.show()

max_frames = int(max(max(pred_fps), max(gt_fps)))

plt.hist(pred_fps, bins=range(0, max_frames, 10), alpha=0.5, color='red', label='Predicted')
plt.hist(gt_fps, bins=range(0, max_frames, 10), alpha=0.5, color='blue',label='Ground Truth')
plt.show()

print("Sperm Per Frame Pred - Mean: ", np.mean(pred_spf), ", Std: ", np.std(pred_spf), ", Normalized Mean: ", np.mean(pred_spf)/max_sperm, ", Normalized Std: ", np.std(pred_spf)/max_sperm)
print("Sperm Per Frame GT - Mean: ", np.mean(gt_spf), ", Std: ", np.std(gt_spf), ", Normalized Mean: ", np.mean(gt_spf)/max_sperm, ", Normalized Std: ", np.std(gt_spf)/max_sperm)
print("Frames Per Sperm Pred - Mean: ", np.mean(pred_fps), ", Std: ", np.std(pred_fps), ", Normalized Mean: ", np.mean(pred_fps)/max_frames, ", Normalized Std: ", np.std(pred_fps)/max_frames)
print("Frames Per Sperm GT - Mean: ", np.mean(gt_fps), ", Std: ", np.std(gt_fps), ", Normalized Mean: ", np.mean(gt_fps)/max_frames, ", Normalized Std: ", np.std(gt_fps)/max_frames)



### Labeled Metrics

In [23]:
from  scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

# Use Hungarian Algorithm to find best track matches between pred and gt in each frame
labels_ref = []
labels_comp = []
mapped_ref = []
mapped_comp = []

# For each frame
for f in range(0, pred['frame'].max() + 1):

    mapped_ref_frame = []
    mapped_comp_frame = []

    # Get the labels in the frame
    labels_ref_frame = gt[gt['frame'] == f]['sperm'].values
    labels_comp_frame = pred[pred['frame'] == f]['sperm'].values

    # Get the centroids in the frame
    ref_centroids = gt[gt['frame'] == f][['x', 'y']].values
    comp_centroids = pred[pred['frame'] == f][['x', 'y']].values

    # Compute the distance matrix
    dist_matrix = cdist(ref_centroids, comp_centroids)

    #import matplotlib.pyplot as plt
    #plt.imshow(dist_matrix);plt.show()
    #print(np.amin(dist_matrix))

    # Use Hungarian Algorithm to find best matches
    row_ind, col_ind = linear_sum_assignment(dist_matrix)

    #for r, c in zip(row_ind, col_ind):
    #    print(r,c)

    # Save the matches
    for r, c in zip(row_ind, col_ind):
        mapped_ref_frame.append(labels_ref_frame[r])
        mapped_comp_frame.append(labels_comp_frame[c])
    
        #labels_ref_frame.append(ref_labels[r])
        #labels_comp_frame.append(comp_labels[c])
        #mapped_ref_frame.append(gt[(gt['frame'] == f) & (gt['sperm'] == ref_labels[r])]['sperm'].values[0])
        #mapped_comp_frame.append(pred[(pred['frame'] == f) & (pred['sperm'] == comp_labels[c])]['sperm'].values[0])

    labels_ref.append(labels_ref_frame)
    labels_comp.append(labels_comp_frame)
    mapped_ref.append(mapped_ref_frame)
    mapped_comp.append(mapped_comp_frame)

traj = {}
traj['labels_ref'] = labels_ref
traj['labels_comp'] = labels_comp
traj['mapped_ref'] = mapped_ref
traj['mapped_comp'] = mapped_comp

#print(labels_ref)
#print(labels_comp)
#print(mapped_ref)
#print(mapped_comp)

In [ ]:
# Create track data needed for calculation (Label, Birth Frame, Death Frame, 0 (no parent))

ref_tracks = []

for s in range(0, gt['sperm'].max() + 1):
    cur_frames = gt[gt['sperm'] == s]['frame'].values
    if len(cur_frames) > 0:
        cur_birth = np.amin(cur_frames)
        cur_death = np.amax(cur_frames)
        cur_parent = 0
        cur_track = [s, cur_birth, cur_death, cur_parent]
        ref_tracks.append(cur_track)

ref_tracks = np.array(ref_tracks)

comp_tracks = []

for s in range(0, pred['sperm'].max() + 1):
    cur_frames = pred[pred['sperm'] == s]['frame'].values
    if len(cur_frames) > 0:
        cur_birth = np.amin(cur_frames)
        cur_death = np.amax(cur_frames)
        cur_parent = 0
        cur_track = [s, cur_birth, cur_death, cur_parent]
        comp_tracks.append(cur_track)

comp_tracks = np.array(comp_tracks)


In [ ]:
from ctc_metrics.utils.representations import merge_tracks, count_acyclic_graph_correction_operations

new_tracks, new_labels, new_mapped = merge_tracks(
    ref_tracks, traj["labels_ref"], traj["mapped_ref"])
traj["ref_tracks_merged"] = new_tracks
traj["labels_ref_merged"] = new_labels
traj["mapped_ref_merged"] = new_mapped
new_tracks, new_labels, new_mapped = merge_tracks(
    comp_tracks, traj["labels_comp"], traj["mapped_comp"])
traj["comp_tracks_merged"] = new_tracks
traj["labels_comp_merged"] = new_labels
traj["mapped_comp_merged"] = new_mapped

graph_operations = \
            count_acyclic_graph_correction_operations(
                ref_tracks, comp_tracks,
                traj["labels_ref"], traj["labels_comp"],
                traj["mapped_ref"], traj["mapped_comp"]
            )

print(graph_operations)

In [30]:
from ctc_metrics.metrics import (
    valid, det, seg, tra, ct, tf, bc, cca, mota, hota, idf1, chota, mtml, faf,
    op_ctb, op_csb, bio, op_clb, lnk
)

results = {}
results["DET"] = det(**graph_operations)
_tra, _aogm, _aogm0 = tra(**graph_operations)
results["TRA"] = _tra
results["AOGM"] = _aogm
results["AOGM_0"] = _aogm0
for key in ("NS", "FN", "FP", "ED", "EA", "EC"):
    results[f"AOGM_{key}"] = graph_operations[key]

results["LNK"] = lnk(**graph_operations)

results["CT"] = ct(
            comp_tracks, ref_tracks,
            traj["labels_ref"], traj["mapped_ref"], traj["mapped_comp"])

results["TF"] = tf(
    ref_tracks,
    traj["labels_ref"], traj["mapped_ref"], traj["mapped_comp"])


results.update(mota(
    traj["labels_ref_merged"], traj["labels_comp_merged"],
    traj["mapped_ref_merged"], traj["mapped_comp_merged"]))

results.update(hota(
    traj["labels_ref_merged"], traj["labels_comp_merged"],
    traj["mapped_ref_merged"], traj["mapped_comp_merged"]))

results.update(idf1(
    traj["labels_ref_merged"], traj["labels_comp_merged"],
    traj["mapped_ref_merged"], traj["mapped_comp_merged"]))


In [ ]:
for key,val in results.items():
    print(key, val)